In [ ]:
from __future__ import annotations

import os 
import numpy as np 
import pandas as pd

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset

from math import ceil
from PIL import Image
from pathlib import Path
from sklearn.metrics import accuracy_score

from mmpfn.datasets.petfinder import PetfinderDataset

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [ ]:
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/petfinder")
dataset = PetfinderDataset(data_path)
_ = dataset.get_images()
_ = dataset.get_embeddings(multimodal_type='all')

Load embeddings from embeddings/pad_ufes_20/pad_ufes_20_cls.pt


/home/wall/works/research/MultiModalPFN/mmpfn/datasets/pad_ufes_20.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.embeddings = torch.load(path)


In [3]:
class ClassifierHead(nn.Module):
	def __init__(self, in_dim:int, n_classes:int, h_dim:int, p_drop:float):
		super().__init__()
		self.classifier = nn.Sequential(
			nn.LayerNorm(in_dim),
			nn.Linear(in_dim, h_dim),
			nn.GELU(),
			nn.Dropout(p=p_drop),
			nn.Linear(h_dim, n_classes),
		)
	def forward(self, x):
		return self.classifier(x)

In [4]:
accuracy_scores = []
n_epochs = 200
for seed in range(10):
    torch.manual_seed(seed)

    train_len = int(len(dataset) * 0.8)
    valid_len = int(train_len * 0.1)
    test_len = len(dataset) - train_len - valid_len
 
    train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_len, valid_len, test_len])
 
    y_train = train_dataset.dataset.y[train_dataset.indices]
    y_valid = valid_dataset.dataset.y[valid_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    
    image_train = train_dataset.dataset.embeddings[train_dataset.indices]
    image_valid = valid_dataset.dataset.embeddings[valid_dataset.indices]
    image_test = test_dataset.dataset.embeddings[test_dataset.indices]

    n_classes = len(set(y_train))

    torch.cuda.empty_cache()

    y_train = torch.from_numpy(y_train)
    y_valid = torch.from_numpy(y_valid)
    y_test = torch.from_numpy(y_test)

    dataset_train = TensorDataset(image_train, y_train)
    dataset_valid = TensorDataset(image_valid, y_valid)
    dataset_test = TensorDataset(image_test, y_test)

    batch_size = 16
    dataloader_train = DataLoader(dataset_train, batch_size = batch_size, shuffle=True)
    dataloader_valid = DataLoader(dataset_valid, batch_size = 1, shuffle=True)
    dataloader_test = DataLoader(dataset_test, batch_size = 1, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ClassifierHead(
        in_dim = image_train.shape[-1], 
        n_classes = n_classes,
        h_dim = int(image_train.shape[-1] / 2),
        p_drop=0.1,
    ).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    best_val_acc = 0.
    best_model_path = "best_header.pth"

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0.
        for images, ys in dataloader_train:
            optimizer.zero_grad()
            preds = model(images.to(device))
            loss = criterion(preds, ys.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, ys in dataloader_valid:
                preds = model(images.to(device))
                predicted = preds.argmax(dim=1)
                correct += (predicted.cpu() == ys).sum().item()
                total += ys.size(0)
                val_acc = correct / total
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)

    best_model = ClassifierHead(
        in_dim=image_train.shape[-1],
        n_classes=n_classes,
        h_dim=int(image_train.shape[-1]/2),
        p_drop=0,
    ).to(device)
    best_model.load_state_dict(torch.load(best_model_path))

    best_model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, ys in dataloader_test:
            preds = model(images.to(device))
            predicted = preds.argmax(dim=1)
            correct += (predicted.cpu() == ys).sum().item()
            total += ys.size(0)
    test_acc = correct / total

    print("accuracy", test_acc)
    accuracy_scores.append(test_acc)


/tmp/ipykernel_6485/2644909814.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))


accuracy 0.6823104693140795


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.8380434782608696
Std Accuracy: 0.01706902477604662
